In [1]:
from __future__ import print_function
from __future__ import division
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from subprocess import check_output
print(check_output(["ls"]).decode("utf8"))

Untitled.ipynb
data



In [3]:
import torch
import sys
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [4]:
from sklearn import cross_validation
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.cross_validation import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split

/Users/kevinlu/miniconda3/envs/pytorch/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)

import numpy
import numpy as np

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

import pandas
import pandas as pd

import logging
handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)

__Python VERSION: 3.6.2 |Continuum Analytics, Inc.| (default, Jul 20 2017, 13:14:59) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
__pyTorch VERSION: 0.2.0+1199e3d


In [6]:
# !pip install psutil
import psutil
import os

In [7]:
def cpu_stats():
    print(sys.version)
    print(psutil.cpu_percent())
    print(psutil.virtual_memory())  # physical memory usage
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
    print('memory GB:', memoryUse)
    

In [8]:
cpu_stats()

3.6.2 |Continuum Analytics, Inc.| (default, Jul 20 2017, 13:14:59) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
33.4
svmem(total=8589934592, available=1911738368, percent=77.7, used=6488051712, free=30879744, active=2101460992, inactive=1880858624, wired=2505732096)
memory GB: 0.1139373779296875


In [9]:
lgr.info("USE CUDA=" + str (use_cuda))

INFO:__main__:USE CUDA=False


### Data Preprocessing`

In [10]:
# fix seed
seed=17*19
np.random.seed(seed)
torch.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed(seed)

#### Data Params

In [11]:
TARGET_VAR= 'target'
BASE_FOLDER = 'data'

#### Shuffle

In [12]:
data = pd.read_json(f'{BASE_FOLDER}/train.json')

In [13]:
data.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [14]:
import random
from datetime import datetime
random.seed(datetime.now())
from sklearn.utils import shuffle
data = shuffle(data) # otherwise same validation set each time!

In [15]:
# Reshape our flattened image
data['band_1'] = data['band_1'].apply(lambda x: np.array(x).reshape(75, 75))
data['band_2'] = data['band_2'].apply(lambda x: np.array(x).reshape(75, 75))
data['inc_angle'] = pd.to_numeric(data['inc_angle'], errors='coerce')

In [16]:
band_1 = np.concatenate([im for im in data['band_1']]).reshape(-1, 75, 75)
band_2 = np.concatenate([im for im in data['band_2']]).reshape(-1, 75, 75)

# Batch, Channel, Height, Width
full_img = np.stack([band_1, band_2], axis=1)

In [17]:
# Convert the np arrays into the correct dimention and type
# Note that BCEloss requires Float in X as well as in y
def XnumpyToTensor(x_data_np):
    x_data_np = np.array(x_data_np, dtype=np.float32)        

    if use_cuda:
        lgr.info ("Using the GPU")    
        X_tensor = (torch.from_numpy(x_data_np).cuda()) # Note the conversion for pytorch    
    else:
        lgr.info ("Using the CPU")
        X_tensor = (torch.from_numpy(x_data_np)) # Note the conversion for pytorch
        
    print(f"x_data_np.shape: {x_data_np.shape}")
    print(f"X_tensor.size(): {X_tensor.shape}")
    return X_tensor


# Convert the np arrays into the correct dimention and type
# Note that BCEloss requires Float in X as well as in y
def YnumpyToTensor(y_data_np):    
    y_data_np=y_data_np.reshape((y_data_np.shape[0],1)) # Must be reshaped for PyTorch!

    if use_cuda:
        lgr.info ("Using the GPU")            
        Y_tensor = (torch.from_numpy(y_data_np)).type(torch.FloatTensor).cuda()  # BCEloss requires Float        
    else:
        lgr.info ("Using the CPU")        
        Y_tensor = (torch.from_numpy(y_data_np)).type(torch.FloatTensor)  # BCEloss requires Float        

    print(f"y_data_np.shape: {y_data_np.shape}")
    print(f"Y_tensor.size(): {Y_tensor.shape}")
    return Y_tensor

In [18]:
class FullTrainingDataset(torch.utils.data.Dataset):
    def __init__(self, full_ds, offset, length):
        self.full_ds = full_ds
        self.offset = offset
        self.length = length
        assert len(full_ds)>=offset+length, Exception("Parent Dataset not long enough")
        super(FullTrainingDataset, self).__init__()
        
    def __len__(self):        
        return self.length
    
    def __getitem__(self, i):
        return self.full_ds[i+self.offset]

In [19]:
def train_test_split(dataset, val_share=0.11):
    val_offset = int(len(dataset)*(1-val_share))
    print("Offset:" + str(val_offset))
    return FullTrainingDataset(dataset, 0, val_offset),\
        FullTrainingDataset(dataset, val_offset, len(dataset)-val_offset)

In [20]:
train_imgs = XnumpyToTensor(full_img)
print()
train_targets = YnumpyToTensor(data['is_iceberg'].values)

INFO:__main__:Using the CPU
INFO:__main__:Using the CPU


x_data_np.shape: (1604, 2, 75, 75)
X_tensor.size(): torch.Size([1604, 2, 75, 75])

y_data_np.shape: (1604, 1)
Y_tensor.size(): torch.Size([1604, 1])


In [21]:
from torch.utils.data import TensorDataset, DataLoader
dset_train = TensorDataset(train_imgs, train_targets)

In [22]:
train_ds, val_ds = train_test_split(dset_train)

Offset:1427


In [23]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=False,
                                            num_workers=1)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=1)

### Define the network

In [29]:
class BasicBlock(nn.Module):
    def __init__(self, in_channel, out_channel, stride, dropout_prob=0.0):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_channel)
        self.relu1 = torch.nn.LeakyReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=5, stride=stride,
                              padding=1, bias=False)
        self.maxpool1=torch.nn.MaxPool2d(2,2)
        self.avgpool=torch.nn.AvgPool2d(2,2)
        self.dropout_prob = dropout_prob
        
    def forward(self, x):
        x = self.conv1(self.relu1(self.bn1(x)))
        x=self.maxpool1(x)
        x=self.avgpool(x)
        if self.dropout_prob > 0:
            x = F.dropout(x, p=self.dropout_prob, training=self.training)
        return x

class NetworkBlock(nn.Module):
    def __init__(self, n_layers, in_channels, out_channels, block, stride, dropout_prob=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_channels, out_channels, n_layers, stride, dropout_prob)
        
    def _make_layer(self, block, in_channels, out_channels, n_layers, stride, dropout_prob):
        layers = []
        for i in range(int(n_layers)):
            layers.append(block(i == 0 and in_channels or out_channels, out_channels, 
            i == 0 and stride or 1, dropout_prob))
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.layer(x)

In [30]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
         # 1st conv before any network block
        self.conv0 = torch.nn.Sequential(
            nn.Conv2d(2, 16, kernel_size=3, stride=1,padding=2, bias=False)
        )
        
        block = BasicBlock
        # 1st block
        self.block1 = NetworkBlock(1, 16, 56, block, 1, 0.5)
        # self.block2 = NetworkBlock(1, 64, 128, block, 1, 0.2)
        # self.block3 = NetworkBlock(1, 64, 128, block, 1, 0.2)
        
        self.features = nn.Sequential(
            self.conv0,
            self.block1,
            # self.block2,
            # self.block3,
        )
        self.classifier = torch.nn.Sequential(
            nn.Linear(18144, 1),
        )
        
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        x = self.sig(x)
        return x


model = Net()
print(model)

Net (
  (conv0): Sequential (
    (0): Conv2d(2, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), bias=False)
  )
  (block1): NetworkBlock (
    (layer): Sequential (
      (0): BasicBlock (
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
        (relu1): LeakyReLU (0.01, inplace)
        (conv1): Conv2d(16, 56, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1), bias=False)
        (maxpool1): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
        (avgpool): AvgPool2d (size=2, stride=2, padding=0, ceil_mode=False, count_include_pad=True)
      )
    )
  )
  (features): Sequential (
    (0): Sequential (
      (0): Conv2d(2, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), bias=False)
    )
    (1): NetworkBlock (
      (layer): Sequential (
        (0): BasicBlock (
          (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
          (relu1): LeakyReLU (0.01, inplace)
          (conv1): Conv2d(16, 56, kernel_size=(5, 5), stride=(1, 

### Loss and optimizer

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [35]:
loss_func=torch.nn.BCELoss()
num_epoches=5
# NN params
LR = 0.0005
MOMENTUM= 0.95
optimizer = torch.optim.Adam(model.parameters(), lr=LR,weight_decay=5e-5) #  L2 regularization

In [36]:
criterion = loss_func
all_losses = []
val_losses = []


# if __name__ == '__main__':

for epoch in range(num_epoches):
    print('Epoch {}'.format(epoch + 1))
    print('*' * 5 + ':')
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):

        img, label = data
        if use_cuda:
            img, label = Variable(img.cuda(async=True)), Variable(label.cuda(async=True))  # On GPU
        else:
            img, label = Variable(img), Variable(
                label)  # RuntimeError: expected CPU tensor (got CUDA tensor)

        out = model(img)
        loss = criterion(out, label)
        running_loss += loss.data[0] * label.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            all_losses.append(running_loss / (batch_size * i))
            print('[{}/{}] Loss: {:.6f}'.format(
                epoch + 1, num_epoches, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))

    print('Finish {} epoch, Loss: {:.6f}'.format(epoch + 1, running_loss / (len(train_ds))))

    model.eval()
    eval_loss = 0
    eval_acc = 0
    for data in val_loader:
        img, label = data

        if use_cuda:
            img, label = Variable(img.cuda(async=True), volatile=True),
            Variable(label.cuda(async=True), volatile=True)  # On GPU
        else:
            img = Variable(img, volatile=True)
            label = Variable(label, volatile=True)

        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)

    print('VALIDATION Loss: {:.6f}'.format(eval_loss / (len(val_ds))))
    val_losses.append(eval_loss / (len(val_ds)))
    print()

torch.save(model.state_dict(), './cnn.pth')

Epoch 1
*****:


KeyboardInterrupt: 